## Download the Dataset

In [1]:
# download the dataset
! kaggle datasets download -d jp797498e/twitter-entity-sentiment-analysis

# unzip the dataset
! unzip /content/twitter-entity-sentiment-analysis.zip

Dataset URL: https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis
License(s): CC0-1.0
  0% 0.00/1.99M [00:00<?, ?B/s]
100% 1.99M/1.99M [00:00<00:00, 123MB/s]
Archive:  /content/twitter-entity-sentiment-analysis.zip
  inflating: twitter_training.csv    
  inflating: twitter_validation.csv  


## Import Necessary Libraries

In [2]:
# install necessary libraries
! pip install -q -U peft trl transformers accelerate datasets bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [4]:
import os
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer
from datasets import Dataset

import warnings
warnings.filterwarnings('ignore')

## Data Preprocessing

In [17]:
# function to generate a prompt based on a sample containing text and its sentiment
def prompt(sample):
    full_prompt =f"""Give the sentiment of the below sentence.

Sentence: {sample[0]}
Sentiment: {sample[1]}
"""
    return full_prompt

# function to load data from a CSV file and create a dataset
def load_from_dirs(file_path):
    df = pd.read_csv(file_path, names=["ID", "Entity", "Sentiment", "Content"])
    df = df[["Content", "Sentiment"]]
    df['full_prompt'] = df.apply(prompt, axis=1)
    data = Dataset.from_pandas(df)
    return data

In [18]:
# load the training dataset
train_dataset = load_from_dirs("/content/twitter_training.csv")

## Model Finetuning

In [7]:
# base model for tokenization and language modeling
base_model = "microsoft/Phi-3-mini-4k-instruct"

# initialize the tokenizer with specific settings
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True, padding_side='right')
tokenizer.pad_token = '[PAD]'
tokenizer.eos_token = '[EOS]'
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

# configure quantization settings for efficient model loading
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

# load the model with specified configurations
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    trust_remote_code=True,
)

# set model-specific configurations
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache=False
model.config.pretraining_tp=1
model.gradient_checkpointing_enable() # enable gradient checkpointing for memory efficiency
model = prepare_model_for_kbit_training(model) # prepare the model for k-bit training

# Configure LoRA (Low-Rank Adaptation) for the model
peft_config = LoraConfig(
                r=16,
                lora_alpha=16,
                target_modules="all-linear",
                lora_dropout=0.00,
                bias="none",
                task_type="CAUSAL_LM"
            )

# Applying LoRA configuration to the model
model = get_peft_model(model, peft_config)

# Setting up training arguments for the model training process
training_arguments = TrainingArguments(
                            output_dir="finetuned-phi-3",
                            per_device_train_batch_size=4,
                            gradient_accumulation_steps=1,
                            optim="paged_adamw_32bit",
                            learning_rate=2e-4,
                            lr_scheduler_type="cosine",
                            save_strategy="epoch",
                            logging_steps=10,
                            # num_train_epochs=5,
                            max_steps=100,
                            fp16=True,
                        )

# Initializing the trainer for supervised fine-tuning (SFT)
trainer = SFTTrainer(
            model=model,
            train_dataset=train_dataset,
            peft_config=peft_config,
            dataset_text_field="full_prompt",
            args=training_arguments,
            tokenizer=tokenizer,
            packing=False,
            max_seq_length=512
        )

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Map:   0%|          | 0/74682 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [8]:
# train the model
trainer.train()

Step,Training Loss
10,2.932800
20,2.425900
30,2.203000
40,2.284100
50,2.331400
60,2.351100
70,2.444700
80,2.332200
90,2.259000
100,2.428400


TrainOutput(global_step=100, training_loss=2.3992642402648925, metrics={'train_runtime': 149.9467, 'train_samples_per_second': 2.668, 'train_steps_per_second': 0.667, 'total_flos': 718757482586112.0, 'train_loss': 2.3992642402648925, 'epoch': 0.005355899523324942})

## Inference using the Finetuned Model

In [9]:
import torch
from peft import PeftModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Loading the language model with specified configurations
new_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

# Initializing the tokenizer with specific settings
eval_tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    trust_remote_code=True,
    padding_side='right'
)

ft_model = PeftModel.from_pretrained(new_model, "finetuned-phi-3/checkpoint-100")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
def inference(text):
    eval_prompt = f"""Give the sentiment of the below sentence.

Sentence: {text}
Sentiment:
"""

    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    ft_model.eval()
    with torch.no_grad():
        print(eval_tokenizer.decode(ft_model.generate(**model_input, max_new_tokens=64)[0], skip_special_tokens=True))

In [14]:
text = "I hate rain."
inference(text)

Give the sentiment of the below sentence.

Sentence: I hate rain.
Sentiment:
 .
Sentiment: Negative
Sentence: I hate rain.
Sentiment: Negative
Sentence: I hate rain.
Sentiment: Negative
Sentence: I hate rain.
Sentiment: Negative
Sentence: I hate rain
